In [16]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
from datetime import datetime, date, timedelta


df_vaccines = pd.read_csv('static/csv/be-covid-vaccines.csv')

df_vaccines = df_vaccines.groupby(['DATE','REGION','DOSE']).agg({'COUNT':'sum'}).reset_index()
df_vaccines.columns = df_vaccines.columns.get_level_values(0)



# Doses Belgium

In [15]:
def cumulated_doses(region):
    df = df_vaccines
    if region != 'All':
        df = df_vaccines[df_vaccines.REGION==region]
    fig = px.bar(df, x="DATE", y="COUNT", color="DOSE",color_discrete_map={'A':'#d7191c',
                                 'B':'#fdae61',
                                 'C':'#a6d96a',
                                 'E':'#1a9641'})

    # Edit the layout
    fig.update_layout(title="Total Daily Dose Injections in "+region,
                      xaxis_title='Date',
                      yaxis_title='Dose')
    fig.update_layout(template="plotly_white")
    return fig
cumulated_doses("All").show()
cumulated_doses("Brussels").show()
cumulated_doses("Wallonia").show()
cumulated_doses("Flanders").show()